In [ ]:
import pandas as pd
from thunderpack import ThunderReader


In [ ]:
all_data= pd.read_csv('/home/bram/Project_PD/afeaturematrix/ALLHOSPITALS_NEW_DATA.csv')

In [ ]:
sample= pd.read_csv ('/home/bram/Project_PD/afeaturematrix/ALLHOSPITALS_NEW_DATA.csv')#[['BDSPPatientID'],['CreateDate']]
print(sample.columns)
get_date=sample[['BDSPPatientID','CreateDate']]
print(get_date)


In [ ]:
print(all_data['BDSPPatientID'].nunique())

In [ ]:
BIDMC=['BIDMC']
MGB=['MGB']
STF=['STF']

In [ ]:
BIDMC_df= all_data[all_data['hospital'].isin(BIDMC)]
MGB_df= all_data[all_data['hospital'].isin(MGB)]
STF_df= all_data[all_data['hospital'].isin(STF)]

In [ ]:
print(len(BIDMC_df))
print(len(MGB_df))
print(len(STF_df))

In [ ]:
BIDMC_PATIENTS=BIDMC_df['BDSPPatientID'].to_list()
MGB_PATIENTS=MGB_df['BDSPPatientID'].to_list()
STF_PATIENTS=STF_df['BDSPPatientID'].to_list()

In [ ]:
print(len(BIDMC_PATIENTS))
print(len(MGB_PATIENTS))
print(len(STF_PATIENTS))

In [ ]:
reader_ICD = ThunderReader('/home/bram/internal_expansion/HOSPITALS/BIDMC/demographics')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)



In [ ]:
df_BIDMCplpl= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/BIDMC/BIDMC_SAMPLE/SAMPLE_ICDPLMEDPL_BIDMC.csv')
df_BIDMCplmi= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/BIDMC/BIDMC_SAMPLE/SAMPLE_ICDPLMEDMI_BIDMC.csv')
df_BIDMCmipl= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/BIDMC/BIDMC_SAMPLE/SAMPLE_ICDMIMEDPL_BIDMC.csv')
df_BIDMCmimi= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/BIDMC/BIDMC_SAMPLE/SAMPLE_ICDMIMEDMI_BIDMC.csv')

In [ ]:
BIDMC_PLPL= set(df_BIDMCplpl['BDSPPatientID'])
BIDMC_PLMI= set(df_BIDMCplmi['BDSPPatientID'])
BIDMC_MIPL= set(df_BIDMCmipl['BDSPPatientID'])
BIDMC_MIMI= set(df_BIDMCmimi['BDSPPatientID'])

In [ ]:
# missingdatebirth=pd.read.csv('')

In [ ]:
# missing_info_bidmc=pd.read_csv('/home/bram/internal_expansion/HOSPITALS/BIDMC/demographics_missing/bidmc_missing_demographics_bidmc_20240719.csv')
# print(missing_info_bidmc)

In [ ]:
# Initialize the list to collect DataFrames
dem_dfs_initial = []

# Loop through each partition and collect DataFrames
for i in range(1, len(all_keys_ICD) + 1):
    dem_df_initial_df = reader_ICD[f'patient_partition_{i}']
    
    # Append the DataFrame to the list
    dem_dfs_initial.append(dem_df_initial_df)

# Concatenate all initial DataFrames into a single DataFrame
dem_df_bidmc = pd.concat(dem_dfs_initial, axis=0, ignore_index=True)
print(dem_df_bidmc)

In [ ]:
print(dem_df_bidmc['BDSPPatientID'].nunique())

In [ ]:
nan_count = dem_df_bidmc['BDSPPatientID'].isna().sum()

print(f"Number of NaN values in 'column_name': {nan_count}")

In [ ]:
filtered_BIDMC_df= dem_df_bidmc[dem_df_bidmc['BDSPPatientID'].isin(BIDMC_PATIENTS)]

In [ ]:
# filtered_BIDMC_missing_df= missing_info_bidmc[missing_info_bidmc['BDSPPatientID'].isin(BIDMC_PATIENTS)]
# print(filtered_BIDMC_missing_df['BDSPPatientID'].nunique())

# filt_missing=filtered_BIDMC_missing_df[['BDSPPatientID','PatientRace', 'EthnicGroupDSC', 'MaritalStatusDSC', 'ReligionDSC', 'LanguageDSC', 'SexDSC', 'BDSPLastModifiedDTS']]

In [ ]:
print(filtered_BIDMC_df)

In [ ]:
filtered_BIDMC_df

In [ ]:
get_date['CreateDate'] = pd.to_datetime(get_date['CreateDate'])
filtered_BIDMC_df = filtered_BIDMC_df[["BDSPPatientID", "DateOfBirth", "PatientRace", "SexDSC",]]
filtered_BIDMC_df['DateOfBirth'] = pd.to_datetime(filtered_BIDMC_df['DateOfBirth'])
filtered_BIDMC_df = filtered_BIDMC_df.merge(get_date, on='BDSPPatientID', how='left')
print(filtered_BIDMC_df)


In [ ]:
filtered_BIDMC_df['Age'] = filtered_BIDMC_df.apply(lambda row: (row['CreateDate'] - row['DateOfBirth']).days / 365.25, axis=1)

In [ ]:
plpl_demos = filtered_BIDMC_df[filtered_BIDMC_df['BDSPPatientID'].isin(BIDMC_PLPL)]
plmi_demos = filtered_BIDMC_df[filtered_BIDMC_df['BDSPPatientID'].isin(BIDMC_PLMI)]
mipl_demos = filtered_BIDMC_df[filtered_BIDMC_df['BDSPPatientID'].isin(BIDMC_MIPL)]
mimi_demos = filtered_BIDMC_df[filtered_BIDMC_df['BDSPPatientID'].isin(BIDMC_MIMI)]
print(len(mimi_demos))

In [ ]:
print(filtered_BIDMC_df['PatientRace'].unique())

In [ ]:
race_mapping = {
    '': 'Unavailable',
    'BLACK/AFRICAN AMERICAN': 'Black',
    # 'AMERICAN INDIAN/ALASKA NATIVE': 'Other',
    'UNABLE TO OBTAIN': 'Unavailable',
    'WHITE': 'White',
    'PREFER NOT TO SAY': 'Unavailable',
    'UNKNOWN/NOT SPECIFIED': 'Unavailable',
    'OTHER RACE': 'Other',
    'DECLINED TO ANSWER': 'Unavailable',
    # 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Other',
    # np.nan: 'Unavailable',
    'ASIAN': 'Asian'
}

In [ ]:
# Apply the mapping to consolidate race categories
plpl_demos['ConsolidatedRace'] = plpl_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = plpl_demos['Age'].mean()
std_dev_age = plpl_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = plpl_demos['SexDSC'].value_counts()
sex_percentage = plpl_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = plpl_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = plpl_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = plpl_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = plpl_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('BIDMC_ICD+MED+.csv', index=False)

In [ ]:
# Apply the mapping to consolidate race categories
plmi_demos['ConsolidatedRace'] = plmi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = plmi_demos['Age'].mean()
std_dev_age = plmi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = plmi_demos['SexDSC'].value_counts()
sex_percentage = plmi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = plmi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = plmi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = plmi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = plmi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD+MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('BIDMC_ICD+MED-.csv', index=False)


In [ ]:
# Apply the mapping to consolidate race categories
mimi_demos['ConsolidatedRace'] = mimi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mimi_demos['Age'].mean()
std_dev_age = mimi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mimi_demos['SexDSC'].value_counts()
sex_percentage = mimi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mimi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mimi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mimi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mimi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD-MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('BIDMC_ICD-MED-.csv', index=False)


In [ ]:
# Apply the mapping to consolidate race categories
mipl_demos['ConsolidatedRace'] = mipl_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mipl_demos['Age'].mean()
std_dev_age = mipl_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mipl_demos['SexDSC'].value_counts()
sex_percentage = mipl_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mipl_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mipl_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mipl_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mipl_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD-MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('BIDMC_ICD-MED+.csv', index=False)

In [ ]:
# Apply the mapping to consolidate race categories
mimi_demos['ConsolidatedRace'] = mimi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mimi_demos['Age'].mean()
std_dev_age = mimi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mimi_demos['SexDSC'].value_counts()
sex_percentage = mimi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mimi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mimi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mimi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mimi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD-MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('BIDMC_ICD-MED-.csv', index=False)


In [ ]:
bi_demos = pd.concat([plpl_demos, plmi_demos, mipl_demos, mimi_demos])

# Apply the mapping to consolidate race categories
bi_demos['ConsolidatedRace'] = bi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = bi_demos['Age'].mean()
std_dev_age = bi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = bi_demos['SexDSC'].value_counts()
sex_percentage = bi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = bi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = bi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = bi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = bi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC All:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")
data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('BIDMC_all.csv', index=False)


In [ ]:
filtered_BIDMC_df.nunique()

In [ ]:
# MISSING_df= filtered_BIDMC_df[~filtered_BIDMC_df['BDSPPatientID'].isin(BIDMC_PATIENTS)]

In [ ]:
# # # Convert the DataFrame column and the list to sets
# df_ids_set = set(filtered_BIDMC_df['BDSPPatientID'])
# check_ids_set = set(BIDMC_PATIENTS)

# # Find the difference
# missing_ids = check_ids_set - df_ids_set

# # Display the result
# print(f"Patient IDs in the list but not in the DataFrame: {missing_ids}")

In [ ]:
# print(len(missing_ids))

In [ ]:
# print(MISSING_df)

In [ ]:
reader_MGB= ThunderReader('/home/bram/internal_expansion/HOSPITALS/MGB/demographics')
all_keys_DEM_MGB = list(reader_MGB.keys())
print(all_keys_DEM_MGB)

In [ ]:
df_mgbplpl_N= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/NEW/SAMPLE_ICDPLMEDPL_MGB.csv')
df_mgbplmi_N= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/NEW/SAMPLE_ICDPLMEDMI_MGB.csv')
df_mgbmipl_N= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/NEW/SAMPLE_ICDMIMEDPL_MGB.csv')
df_mgbmimi_N= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/NEW/SAMPLE_ICDMIMEDMI_MGB.csv')

print(df_mgbplpl_N.columns)

In [ ]:
df_mgbplpl_EL= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/ELIJAH/sample_ICDPLMEDPL_2_COR_MGH_ELIJAH.csv')
df_mgbplmi_EL= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/ELIJAH/SAMPLE_ICDPLMEDMI_MGH_ELIJAH.csv')
df_mgbmipl_EL= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/ELIJAH/SAMPLE_ICDMIMEDPL_MGH_ELIJAH.csv')
df_mgbmimi_EL= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/MGB/MGB_SAMPLE/ELIJAH/SAMPLE_ICDMIMEDMI_MGH_ELIJAH.csv')

In [ ]:

df_mgbplpl_EL= df_mgbplpl_EL.drop(['annot', 'group'], axis=1)
df_mgbplmi_EL= df_mgbplmi_EL.drop(['annot', 'group'], axis=1)
df_mgbmipl_EL= df_mgbmipl_EL.drop(['annot', 'group'], axis=1)
df_mgbmimi_EL= df_mgbmimi_EL.drop(['annot', 'group'], axis=1)

In [ ]:
MGB_PLPL=set( list(df_mgbplpl_N['BDSPPatientID'])+list(df_mgbplpl_EL['BDSPPatientID']))
len(MGB_PLPL)
MGB_PLMI= set(list(df_mgbplmi_N['BDSPPatientID'])+list(df_mgbplmi_EL['BDSPPatientID']))
MGB_MIPL= set(list(df_mgbmipl_N['BDSPPatientID'])+list(df_mgbmipl_EL['BDSPPatientID']))
MGB_MIMI= set(list(df_mgbmimi_N['BDSPPatientID'])+list(df_mgbmimi_EL['BDSPPatientID']))

In [ ]:
# Initialize the list to collect DataFrames
dem_MGB_dfs_initial = []

# Loop through each partition and collect DataFrames
for i in range(1, len(all_keys_DEM_MGB)):
    dem_MGB_initial_df = reader_MGB[f'demographics_partition_{i}']
    
    # Append the DataFrame to the list
    dem_MGB_dfs_initial.append(dem_MGB_initial_df)

# Concatenate all initial DataFrames into a single DataFrame
dem_df_MGB = pd.concat(dem_MGB_dfs_initial, axis=0, ignore_index=True)
print(dem_df_MGB)

In [ ]:
Metadata=reader_MGB['metadata']
print(Metadata)

In [ ]:
filtered_MGB_df= dem_df_MGB[dem_df_MGB['BDSPPatientID'].isin(MGB_PATIENTS)]
print(filtered_MGB_df)
print(filtered_MGB_df['BDSPPatientID'].nunique())

In [ ]:
print(get_date)
df = filtered_MGB_df[["BDSPPatientID", "DateOfBirth", "PatientRace", "SexDSC", "GenderIdentityDSC"]]
df = df.drop_duplicates(subset="BDSPPatientID", keep="first")
get_date['CreateDate'] = pd.to_datetime(get_date['CreateDate'])
df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'])
df = df.merge(get_date, on='BDSPPatientID', how='left')
df['Age'] = df.apply(lambda row: (row['CreateDate'] - row['DateOfBirth']).days / 365.25, axis=1)

In [ ]:
plpl_demos = df[df['BDSPPatientID'].isin(MGB_PLPL)]
plmi_demos = df[df['BDSPPatientID'].isin(MGB_PLMI)]
mipl_demos = df[df['BDSPPatientID'].isin(MGB_MIPL)]
mimi_demos = df[df['BDSPPatientID'].isin(MGB_MIMI)]

In [ ]:
print(df['PatientRace'].unique())

In [ ]:
race_mapping = {
    'White': 'White',
    'Other; White' : 'White',
    'Declined; White' : 'White',
    'Unavailable; White': 'White',
    'Black or African American': 'Black',
    'Black or African American; Unavailable': 'Black',
    'Black or African American; Other': 'Black',
    'Asian': 'Asian',
    'Asian; Other': 'Asian',
    'Declined; Other':'Other',
    None : 'Unavailable',
    'Asian; White': 'Other',
    'Other': 'Other',
    'Unavailable': 'Unavailable',
    'Declined': 'Unavailable',
    'American Indian or Alaska Native': 'Other',
    'American Indian or Alaska Native; White': 'Other',
    'Native Hawaiian or Other Pacific Islander': 'Other'
}

In [ ]:
# Apply the mapping to consolidate race categories
plpl_demos['ConsolidatedRace'] = plpl_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = plpl_demos['Age'].mean()
std_dev_age = plpl_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = plpl_demos['SexDSC'].value_counts()
sex_percentage = plpl_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = plpl_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = plpl_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = plpl_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = plpl_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("MGB ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('MGB_ICD+MED+.csv', index=False)

In [ ]:
# Apply the mapping to consolidate race categories
plmi_demos['ConsolidatedRace'] = plmi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = plmi_demos['Age'].mean()
std_dev_age = plmi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = plmi_demos['SexDSC'].value_counts()
sex_percentage = plmi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = plmi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = plmi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = plmi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = plmi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("MGB ICD+MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('MGB_ICD+MED-.csv', index=False)


In [ ]:
# Apply the mapping to consolidate race categories
mipl_demos['ConsolidatedRace'] = mipl_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mipl_demos['Age'].mean()
std_dev_age = mipl_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mipl_demos['SexDSC'].value_counts()
sex_percentage = mipl_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mipl_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mipl_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mipl_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mipl_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD-MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('MGB_ICD-MED+.csv', index=False)



In [ ]:
# Apply the mapping to consolidate race categories
mimi_demos['ConsolidatedRace'] = mimi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mimi_demos['Age'].mean()
std_dev_age = mimi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mimi_demos['SexDSC'].value_counts()
sex_percentage = mimi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mimi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mimi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mimi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mimi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("BIDMC ICD-MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('MGB_ICD-MED-.csv', index=False)


In [ ]:
MGB_demos = pd.concat([plpl_demos, plmi_demos, mipl_demos, mimi_demos])

# Apply the mapping to consolidate race categories
MGB_demos['ConsolidatedRace'] = MGB_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = MGB_demos['Age'].mean()
std_dev_age = MGB_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = MGB_demos['SexDSC'].value_counts()
sex_percentage = MGB_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = MGB_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = MGB_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = MGB_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = MGB_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("MGB All:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('MGB_all.csv', index=False)


In [ ]:
reader_STF= ThunderReader('/home/bram/internal_expansion/HOSPITALS/Stanford/demographics')
all_keys_DEM_STF = list(reader_STF.keys())
print(all_keys_DEM_STF)

In [ ]:
stf=reader_STF['demographics_partition_1']
print(stf)

In [ ]:
filtered_STF_df= stf[stf['bdsp_patient_id'].isin(STF_PATIENTS)]
print(filtered_STF_df)
print(filtered_STF_df['bdsp_patient_id'].nunique())

In [ ]:
df_STANplpl= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/Stanford/stanford_sample/SAMPLE_ICDPLMEDPL_STAN.csv')
df_STANplmi= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/Stanford/stanford_sample/SAMPLE_ICDPLMEDMI_STAN.csv')
df_STANmipl= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/Stanford/stanford_sample/SAMPLE_ICDMIMEDPL_STAN.csv')
df_STANmimi= pd.read_csv('/home/bram/internal_expansion/HOSPITALS/Stanford/stanford_sample/SAMPLE_ICDMIMEDMI_STAN.csv')

In [ ]:
STF_PLPL= list(df_STANplpl['BDSPPatientID'])
STF_PLMI= list(df_STANplmi['BDSPPatientID'])
STF_MIPL= list(df_STANmipl['BDSPPatientID'])
STF_MIMI= list(df_STANmimi['BDSPPatientID'])
print(len(STF_PLPL))
print(len(STF_PLMI))
print(len(STF_MIPL))
print(len(STF_MIMI))

In [ ]:
print(filtered_STF_df.columns)

In [ ]:
df = filtered_STF_df[["bdsp_patient_id", "gender", "shifted_date_of_birth", "race"]]
new_columns= {'bdsp_patient_id':'BDSPPatientID', 'gender': 'SexDSC', 'shifted_date_of_birth': 'DateOfBirth', 'race': 'PatientRace' }
df = df.rename(columns=new_columns)
print(df)
df = df.drop_duplicates(subset="BDSPPatientID", keep="first")
get_date['CreateDate'] = pd.to_datetime(get_date['CreateDate'])
df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'])
df = df.merge(get_date, on='BDSPPatientID', how='left')
df['Age'] = df.apply(lambda row: (row['CreateDate'] - row['DateOfBirth']).days / 365.25, axis=1)

In [ ]:
plpl_demos = df[df['BDSPPatientID'].isin(STF_PLPL)]
plmi_demos = df[df['BDSPPatientID'].isin(STF_PLMI)]
mipl_demos = df[df['BDSPPatientID'].isin(STF_MIPL)]
mimi_demos = df[df['BDSPPatientID'].isin(STF_MIMI)]

In [ ]:
print(df['PatientRace'].unique())

In [ ]:
race_mapping = {
    'Unknown': 'Unavailable',
    'Black': 'Black',
    # 'AMERICAN INDIAN/ALASKA NATIVE': 'Other',
    # 'UNABLE TO OBTAIN': 'Unavailable',
    'White': 'White',
    # 'PREFER NOT TO SAY': 'Unavailable',
    'Unknown': 'Unavailable',
    'Other': 'Other',
    'Native American': 'Other',
    'Pacific Islander': 'Other',
    # 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Other',
    # np.nan: 'Unavailable',
    'Asian': 'Asian'
}

In [ ]:
# Apply the mapping to consolidate race categories
plpl_demos['ConsolidatedRace'] = plpl_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = plpl_demos['Age'].mean()
std_dev_age = plpl_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = plpl_demos['SexDSC'].value_counts()
sex_percentage = plpl_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = plpl_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = plpl_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = plpl_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = plpl_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("STF ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('STF_ICD+MED+.csv', index=False)

In [ ]:
# Apply the mapping to consolidate race categories
plmi_demos['ConsolidatedRace'] = plmi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = plmi_demos['Age'].mean()
std_dev_age = plmi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = plmi_demos['SexDSC'].value_counts()
sex_percentage = plmi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = plmi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = plmi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = plmi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = plmi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("STF ICD+MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

# Display the metrics
print("STF ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('STF_ICD+MED-.csv', index=False)


In [ ]:
# Apply the mapping to consolidate race categories
mipl_demos['ConsolidatedRace'] = mipl_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mipl_demos['Age'].mean()
std_dev_age = mipl_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mipl_demos['SexDSC'].value_counts()
sex_percentage = mipl_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mipl_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mipl_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mipl_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mipl_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("STF ICD-MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

# Display the metrics
print("STF ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('STF_ICD-MED+.csv', index=False)


In [ ]:
# Apply the mapping to consolidate race categories
mimi_demos['ConsolidatedRace'] = mimi_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = mimi_demos['Age'].mean()
std_dev_age = mimi_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = mimi_demos['SexDSC'].value_counts()
sex_percentage = mimi_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = mimi_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = mimi_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = mimi_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = mimi_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("STF ICD-MED-:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

# Display the metrics
print("STF ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('STF_ICD-MED-.csv', index=False)


In [ ]:
STF_demos = pd.concat([plpl_demos, plmi_demos, mipl_demos, mimi_demos])

# Apply the mapping to consolidate race categories
STF_demos['ConsolidatedRace'] = STF_demos['PatientRace'].apply(
    lambda x: 'Other' if pd.isna(x) else race_mapping.get(x.split(';')[0].strip(), 'Other')
)

# Calculate the metrics with consolidated race categories
average_age = STF_demos['Age'].mean()
std_dev_age = STF_demos['Age'].std()

# Calculate male/female distribution
sex_distribution = STF_demos['SexDSC'].value_counts()
sex_percentage = STF_demos['SexDSC'].value_counts(normalize=True) * 100
sex_na_count = STF_demos['SexDSC'].isna().sum()

# Calculate consolidated race distribution
consolidated_race_distribution = STF_demos['ConsolidatedRace'].value_counts()
consolidated_race_percentage = STF_demos['ConsolidatedRace'].value_counts(normalize=True) * 100
consolidated_race_na_count = STF_demos['ConsolidatedRace'].isna().sum()

# Display the metrics
print("STF All:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

# Display the metrics
print("STF ICD+MED+:")
print(f"Average age: {average_age:.2f} years")
print(f"Standard deviation of age: {std_dev_age:.2f} years")

print("\nSex distribution:")
print(sex_distribution)
print("\nSex percentage:")
print(sex_percentage)
print(f"\nSex NA count: {sex_na_count}")

print("\nConsolidated Race distribution:")
print(consolidated_race_distribution)
print("\nConsolidated Race percentage:")
print(consolidated_race_percentage)
print(f"\nConsolidated Race NA count: {consolidated_race_na_count}")

data = {
    'Metric': [
        'Average Age', 
        'Standard Deviation of Age', 
        'Sex Distribution (Male)', 
        'Sex Distribution (Female)', 
        'Sex NA Count',
        'Consolidated Race Distribution (White)', 
        'Consolidated Race Distribution (Black)', 
        'Consolidated Race Distribution (Asian)',
        'Consolidated Race Distribution (Unavailable)',
        'Consolidated Race Distribution (Other)',
        'Consolidated Race NA Count'
    ],
    'Value': [
        f"{average_age:.2f} years", 
        f"{std_dev_age:.2f} years", 
        sex_distribution.get('Male', 0), 
        sex_distribution.get('Female', 0), 
        sex_na_count, 
        consolidated_race_distribution.get('White', 0), 
        consolidated_race_distribution.get('Black', 0),
        consolidated_race_distribution.get('Asian', 0),
        consolidated_race_distribution.get('Unavailable', 0),
        consolidated_race_distribution.get('Other', 0),
        consolidated_race_na_count
    ],
    'Percentage': [
        '', 
        '', 
        f"{sex_percentage.get('Male', 0):.2f}%", 
        f"{sex_percentage.get('Female', 0):.2f}%", 
        '', 
        f"{consolidated_race_percentage.get('White', 0):.2f}%", 
        f"{consolidated_race_percentage.get('Black', 0):.2f}%",
        f"{consolidated_race_percentage.get('Asian', 0):.2f}%",
        f"{consolidated_race_percentage.get('Unavailable', 0):.2f}%",
        f"{consolidated_race_percentage.get('Other', 0):.2f}%",
        ''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame to check for errors
df.to_csv('STF_all.csv', index=False)
